# VGG16 Discriminator with resnet generator

In [4]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-whp2nn2f
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-whp2nn2f
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101064 sha256=eed56b1dd6923a8b9e72f7eff171d59b2cf944684b36a239164eda9edfbff899
  Stored in directory: /tmp/pip-ephem-wheel-cache-ldfm6ad9/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [5]:
#imports
import sys
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD


from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation


import numpy as np
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from matplotlib import pyplot

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
									Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from keras.datasets import fashion_mnist
from keras.utils import to_categorical


from os import listdir
from numpy import asarray
from numpy import vstack
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from numpy import savez_compressed
import numpy as np
from numpy import load

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization

/opt/conda/lib/python3.7/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
# load and prepare training images
def load_real_samples(filename):
	# load the dataset
	data = load(filename)
	# unpack arrays
	X1, X2 = data['arr_0'], data['arr_1']
	# scale from [0,255] to [-1,1]
	X1 = (X1 - 127.5) / 127.5
	X2 = (X2 - 127.5) / 127.5
	return [X1, X2]

# load image data
dataset = load_real_samples('../input/lego2house/legotohouse.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
image_shape = dataset[0].shape[1:]
print(image_shape)

Loaded (1238, 256, 256, 3) (535, 256, 256, 3)
(256, 256, 3)


In [7]:
# generator a resnet block
def resnet_block(n_filters, input_layer):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# first layer convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# second convolutional layer
	g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	# concatenate merge channel-wise with input layer
	g = Concatenate()([g, input_layer])
	return g



# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=image_shape)
	# c7s1-64
	g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d128
	g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# d256
	g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# R256
	for _ in range(n_resnet):
		g = resnet_block(256, g)
	# u128
	g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# u64
	g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	g = Activation('relu')(g)
	# c7s1-3
	g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
	g = InstanceNormalization(axis=-1)(g)
	out_image = Activation('tanh')(g)
	# define model
	model = Model(in_image, out_image)
	return model

In [8]:
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)

In [9]:
def define_discriminator(image_shape):
    # vgg16
    init = RandomNormal(stddev=0.02)
    
    model = Sequential()
    model.add(Conv2D(64,(3,3),activation='relu',input_shape=image_shape,padding='same'))
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(256,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(512,(3,3),activation='relu',padding='same'))
    model.add(Conv2D(1, (4,4), padding='same', kernel_initializer=init))
    model.compile(loss='mse', optimizer=Adam(lr = 0.0002), loss_weights=[0.5])
    return model

In [10]:
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)

In [11]:
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
	# ensure the model we're updating is trainable
	g_model_1.trainable = True
	# mark discriminator as not trainable
	d_model.trainable = False
	# mark other generator model as not trainable
	g_model_2.trainable = False
	# discriminator element
	input_gen = Input(shape=image_shape)
	gen1_out = g_model_1(input_gen)
	output_d = d_model(gen1_out)
	# identity element
	input_id = Input(shape=image_shape)
	output_id = g_model_1(input_id)
	# forward cycle
	output_f = g_model_2(gen1_out)
	# backward cycle
	gen2_out = g_model_2(input_id)
	output_b = g_model_1(gen2_out)
	# define model graph
	model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
	# define optimization algorithm configuration
	opt = Adam(lr=0.0002, beta_1=0.5)
	# compile model with weighting of least squares loss and L1 loss
	model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
	return model

In [12]:
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)

In [13]:
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
	# define properties of the training run
	n_epochs, n_batch, = 50, 1
	# determine the output square shape of the discriminator
	n_patch = d_model_A.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# prepare image pool for fakes
	poolA, poolB = list(), list()
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
		X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
		X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
		# update fakes from pool
		X_fakeA = update_image_pool(poolA, X_fakeA)
		X_fakeB = update_image_pool(poolB, X_fakeB)
		# update generator B->A via adversarial and cycle loss
		g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
		# update discriminator for A -> [real/fake]
		dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
		dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
		# update generator A->B via adversarial and cycle loss
		g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
		# update discriminator for B -> [real/fake]
		dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
		dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
		# summarize performance
		print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
		# evaluate the model performance every so often
		if (i+1) % (bat_per_epo * 1) == 0:
			# plot A->B translation
			summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
			# plot B->A translation
			summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
		if (i+1) % (bat_per_epo * 5) == 0:
			# save the models
			save_models(i, g_model_AtoB, g_model_BtoA)

In [14]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, patch_shape, patch_shape, 1))
	return X, y

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, dataset, patch_shape):
	# generate fake instance
	X = g_model.predict(dataset)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

# save the generator models to file
def save_models(step, g_model_AtoB, g_model_BtoA):
	# save the first generator model
	filename1 = 'g_model_AtoB_%06d_30.h5' % (step+1)
	g_model_AtoB.save(filename1)
	# save the second generator model
	filename2 = 'g_model_BtoA_%06d_30.h5' % (step+1)
	g_model_BtoA.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))
    

    # update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)

# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, trainX, name, n_samples=5):
	# select a sample of input images
	X_in, _ = generate_real_samples(trainX, n_samples, 0)
	# generate translated images
	X_out, _ = generate_fake_samples(g_model, X_in, 0)
	# scale all pixels from [-1,1] to [0,1]
	X_in = (X_in + 1) / 2.0
	X_out = (X_out + 1) / 2.0
	# plot real images
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + i)
		pyplot.axis('off')
		pyplot.imshow(X_in[i])
	# plot translated image
	for i in range(n_samples):
		pyplot.subplot(2, n_samples, 1 + n_samples + i)
		pyplot.axis('off')
		pyplot.imshow(X_out[i])
	# save plot to file
	filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
	pyplot.savefig(filename1)
	pyplot.close()

# update image pool for fake images
def update_image_pool(pool, images, max_size=50):
	selected = list()
	for image in images:
		if len(pool) < max_size:
			# stock the pool
			pool.append(image)
			selected.append(image)
		elif random() < 0.5:
			# use image, but don't add it to the pool
			selected.append(image)
		else:
			# replace an existing image and use replaced image
			ix = randint(0, len(pool))
			selected.append(pool[ix])
			pool[ix] = image
	return asarray(selected)

In [ ]:
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)

>1, dA[0.496,0.032] dB[0.501,0.008] g[19.254,23.279]
>2, dA[0.474,0.002] dB[0.486,0.002] g[12.279,14.682]
>3, dA[0.422,0.021] dB[0.455,0.023] g[11.400,13.403]
>4, dA[0.287,0.432] dB[0.370,0.084] g[9.523,10.504]
>5, dA[0.248,0.026] dB[0.288,0.084] g[8.856,10.095]
>6, dA[0.346,0.007] dB[0.259,0.071] g[10.054,10.933]
>7, dA[0.387,0.005] dB[0.201,0.171] g[7.745,8.923]
>8, dA[0.361,0.010] dB[0.164,0.108] g[8.965,10.251]
>9, dA[0.307,0.044] dB[0.244,0.080] g[9.489,9.826]
>10, dA[0.227,0.180] dB[0.226,0.066] g[7.209,7.924]
>11, dA[0.113,0.176] dB[0.204,0.105] g[7.280,9.340]
>12, dA[0.264,0.090] dB[0.215,0.134] g[10.877,11.973]
>13, dA[0.203,0.056] dB[0.109,0.155] g[7.998,8.686]
>14, dA[0.178,0.065] dB[0.137,0.088] g[7.988,9.052]
>15, dA[0.251,0.110] dB[0.183,0.127] g[11.901,14.539]
>16, dA[0.161,0.146] dB[0.152,0.120] g[8.025,9.506]
>17, dA[0.147,0.124] dB[0.153,0.088] g[6.707,7.342]
>18, dA[0.163,0.122] dB[0.144,0.114] g[8.338,8.833]
>19, dA[0.228,0.102] dB[0.140,0.124] g[7.942,9.253]
>20, d

>157, dA[0.073,0.026] dB[0.071,0.078] g[7.168,7.836]
>158, dA[0.221,0.007] dB[0.132,0.069] g[7.654,8.359]
>159, dA[0.326,0.003] dB[0.071,0.079] g[5.917,7.180]
>160, dA[0.326,0.006] dB[0.081,0.109] g[5.259,6.067]
>161, dA[0.271,0.009] dB[0.092,0.059] g[4.635,5.607]
>162, dA[0.307,0.023] dB[0.111,0.094] g[4.124,5.161]
>163, dA[0.268,0.046] dB[0.155,0.055] g[5.368,5.465]
>164, dA[0.103,0.073] dB[0.098,0.052] g[6.203,7.097]
>165, dA[0.063,0.256] dB[0.106,0.050] g[7.592,8.079]
>166, dA[0.061,0.231] dB[0.125,0.055] g[5.181,4.939]
>167, dA[0.132,0.091] dB[0.107,0.051] g[4.920,4.642]
>168, dA[0.189,0.059] dB[0.097,0.132] g[6.380,8.547]
>169, dA[0.092,0.047] dB[0.182,0.059] g[6.094,7.328]
>170, dA[0.076,0.072] dB[0.098,0.050] g[6.443,7.079]
>171, dA[0.044,0.072] dB[0.112,0.059] g[5.889,6.179]
>172, dA[0.125,0.086] dB[0.133,0.049] g[4.928,5.698]
>173, dA[0.052,0.164] dB[0.093,0.045] g[5.460,6.149]
>174, dA[0.160,0.081] dB[0.127,0.043] g[6.039,7.388]
>175, dA[0.197,0.077] dB[0.151,0.042] g[6.851,

>312, dA[0.171,0.098] dB[0.138,0.126] g[5.113,5.443]
>313, dA[0.203,0.098] dB[0.051,0.135] g[6.660,7.457]
>314, dA[0.083,0.107] dB[0.074,0.059] g[10.028,12.099]
>315, dA[0.105,0.123] dB[0.095,0.184] g[6.789,7.833]
>316, dA[0.065,0.154] dB[0.092,0.051] g[8.166,9.491]
>317, dA[0.055,0.126] dB[0.093,0.103] g[7.694,9.439]
>318, dA[0.131,0.109] dB[0.113,0.085] g[4.052,4.404]
>319, dA[0.142,0.124] dB[0.101,0.098] g[5.199,5.277]
>320, dA[0.277,0.093] dB[0.187,0.127] g[9.192,13.191]
>321, dA[0.211,0.080] dB[0.075,0.069] g[6.029,5.750]
>322, dA[0.125,0.078] dB[0.105,0.067] g[6.767,8.510]
>323, dA[0.207,0.091] dB[0.063,0.082] g[5.885,6.171]
>324, dA[0.196,0.094] dB[0.102,0.027] g[4.857,4.813]
>325, dA[0.096,0.138] dB[0.097,0.110] g[4.977,5.984]
>326, dA[0.124,0.146] dB[0.069,0.101] g[4.967,5.887]
>327, dA[0.074,0.133] dB[0.113,0.071] g[6.265,7.114]
>328, dA[0.052,0.125] dB[0.060,0.123] g[8.409,10.634]
>329, dA[0.084,0.134] dB[0.133,0.043] g[7.058,7.907]
>330, dA[0.075,0.107] dB[0.104,0.029] g[7.

>467, dA[0.047,0.149] dB[0.077,0.066] g[5.414,5.870]
>468, dA[0.061,0.142] dB[0.084,0.039] g[4.802,4.787]
>469, dA[0.017,0.030] dB[0.123,0.048] g[7.150,8.158]
>470, dA[0.193,0.144] dB[0.325,0.054] g[4.736,4.947]
>471, dA[0.018,0.021] dB[0.066,0.050] g[9.938,11.256]
>472, dA[0.074,0.015] dB[0.233,0.015] g[7.526,8.670]
>473, dA[0.257,0.011] dB[0.186,0.040] g[5.752,6.380]
>474, dA[0.204,0.028] dB[0.112,0.058] g[7.816,9.619]
>475, dA[0.036,0.059] dB[0.134,0.037] g[6.462,7.333]
>476, dA[0.037,0.060] dB[0.084,0.143] g[6.444,8.571]
>477, dA[0.113,0.059] dB[0.066,0.089] g[3.822,4.318]
>478, dA[0.198,0.021] dB[0.081,0.055] g[4.927,5.406]
>479, dA[0.242,0.040] dB[0.147,0.128] g[5.264,5.699]
>480, dA[0.268,0.051] dB[0.210,0.023] g[5.117,5.847]
>481, dA[0.118,0.197] dB[0.156,0.253] g[4.634,5.422]
>482, dA[0.151,0.067] dB[0.040,0.100] g[4.182,4.606]
>483, dA[0.186,0.099] dB[0.113,0.158] g[4.437,5.545]
>484, dA[0.092,0.134] dB[0.090,0.144] g[5.037,5.088]
>485, dA[0.015,0.115] dB[0.059,0.018] g[7.340

>622, dA[0.026,0.199] dB[0.134,0.025] g[6.744,7.204]
>623, dA[0.017,0.097] dB[0.068,0.130] g[5.422,6.270]
>624, dA[0.050,0.055] dB[0.042,0.125] g[6.942,8.672]
>625, dA[0.253,0.051] dB[0.018,0.034] g[5.540,5.917]
>626, dA[0.259,0.019] dB[0.020,0.291] g[3.834,5.228]
>627, dA[0.090,0.024] dB[0.042,0.011] g[6.182,7.961]
>628, dA[0.237,0.016] dB[0.081,0.013] g[3.246,3.757]
>629, dA[0.038,0.037] dB[0.092,0.046] g[6.802,8.349]
>630, dA[0.145,0.066] dB[0.065,0.110] g[4.047,4.346]
>631, dA[0.014,0.183] dB[0.066,0.051] g[6.185,6.907]
>632, dA[0.107,0.063] dB[0.192,0.018] g[4.393,3.982]
>633, dA[0.256,0.029] dB[0.031,0.022] g[6.747,8.867]
>634, dA[0.006,0.034] dB[0.059,0.188] g[5.042,5.610]
>635, dA[0.042,0.076] dB[0.012,0.114] g[8.373,9.952]
>636, dA[0.065,0.066] dB[0.102,0.048] g[5.416,5.939]
>637, dA[0.123,0.052] dB[0.089,0.006] g[5.220,5.556]
>638, dA[0.189,0.124] dB[0.161,0.018] g[5.901,6.591]
>639, dA[0.021,0.150] dB[0.121,0.577] g[5.765,6.355]
>640, dA[0.133,0.132] dB[0.178,0.133] g[5.753,

...

>3753, dA[0.016,0.059] dB[0.042,0.006] g[5.171,5.024]
>3754, dA[0.012,0.018] dB[0.158,0.005] g[3.661,3.116]
>3755, dA[0.185,0.016] dB[0.008,0.270] g[3.612,4.689]
>3756, dA[0.026,0.012] dB[0.001,0.030] g[5.190,5.681]
>3757, dA[0.168,0.028] dB[0.098,0.073] g[5.971,5.955]
>3758, dA[0.029,0.140] dB[0.058,0.009] g[3.935,3.906]
>3759, dA[0.137,0.224] dB[0.025,0.004] g[3.181,3.387]
>3760, dA[0.006,0.106] dB[0.086,0.123] g[5.625,6.660]
>3761, dA[0.005,0.156] dB[0.087,0.164] g[5.132,5.175]
>3762, dA[0.010,0.039] dB[0.032,0.033] g[5.812,5.790]
>3763, dA[0.029,0.006] dB[0.004,0.062] g[6.572,6.763]
>3764, dA[0.073,0.014] dB[0.075,0.002] g[5.341,5.307]
>3765, dA[0.257,0.028] dB[0.248,0.005] g[4.914,5.491]
>3766, dA[0.133,0.031] dB[0.094,0.004] g[5.706,5.605]
>3767, dA[0.046,0.017] dB[0.087,0.008] g[10.773,14.594]
>3768, dA[0.244,0.016] dB[0.063,0.018] g[7.066,8.501]
>3769, dA[0.019,0.096] dB[0.033,0.028] g[3.508,3.690]
>3770, dA[0.008,0.047] dB[0.005,0.189] g[5.584,6.564]
>3771, dA[0.042,0.050] dB[

>3905, dA[0.064,0.044] dB[0.100,0.004] g[3.745,3.555]
>3906, dA[0.154,0.039] dB[0.249,0.013] g[4.910,5.492]
>3907, dA[0.274,0.042] dB[0.088,0.011] g[10.836,14.290]
>3908, dA[0.033,0.071] dB[0.031,0.014] g[3.504,3.560]
>3909, dA[0.035,0.033] dB[0.011,0.027] g[4.368,4.304]
>3910, dA[0.037,0.013] dB[0.004,0.200] g[4.566,5.831]
>3911, dA[0.245,0.053] dB[0.015,0.160] g[4.667,5.596]
>3912, dA[0.020,0.010] dB[0.004,0.009] g[3.242,4.082]
>3913, dA[0.020,0.076] dB[0.019,0.002] g[4.913,4.813]
>3914, dA[0.018,0.020] dB[0.184,0.003] g[4.822,4.803]
>3915, dA[0.040,0.030] dB[0.180,0.002] g[6.008,5.866]
>3916, dA[0.034,0.038] dB[0.108,0.006] g[4.140,4.490]
>3917, dA[0.019,0.139] dB[0.159,0.009] g[4.115,4.135]
>3918, dA[0.012,0.016] dB[0.006,0.108] g[5.655,6.957]
>3919, dA[0.139,0.023] dB[0.026,0.010] g[6.732,7.776]
>3920, dA[0.125,0.013] dB[0.006,0.082] g[2.823,3.678]
>3921, dA[0.043,0.009] dB[0.013,0.133] g[7.088,6.363]
>3922, dA[0.034,0.007] dB[0.006,0.080] g[4.829,5.329]
>3923, dA[0.024,0.136] dB[

>4057, dA[0.003,0.117] dB[0.083,0.003] g[4.871,4.953]
>4058, dA[0.006,0.004] dB[0.011,0.095] g[6.588,6.646]
>4059, dA[0.007,0.012] dB[0.008,0.005] g[4.022,3.805]
>4060, dA[0.007,0.070] dB[0.023,0.034] g[3.833,4.060]
>4061, dA[0.012,0.007] dB[0.007,0.069] g[5.403,6.257]
>4062, dA[0.073,0.011] dB[0.007,0.150] g[4.985,5.997]
>4063, dA[0.007,0.008] dB[0.005,0.214] g[5.132,6.392]
>4064, dA[0.005,0.007] dB[0.019,0.003] g[6.069,7.122]
>4065, dA[0.007,0.008] dB[0.178,0.021] g[4.686,5.367]
>4066, dA[0.118,0.008] dB[0.034,0.024] g[5.702,7.531]
>4067, dA[0.172,0.005] dB[0.165,0.015] g[5.007,4.977]
>4068, dA[0.038,0.019] dB[0.131,0.027] g[6.527,6.399]
>4069, dA[0.011,0.006] dB[0.011,0.012] g[5.358,6.169]
>4070, dA[0.284,0.126] dB[0.003,0.101] g[4.390,5.097]
>4071, dA[0.030,0.096] dB[0.004,0.096] g[3.218,4.426]
>4072, dA[0.025,0.286] dB[0.009,0.022] g[5.996,5.050]
>4073, dA[0.007,0.060] dB[0.036,0.006] g[5.162,4.715]
>4074, dA[0.004,0.013] dB[0.026,0.014] g[4.859,5.584]
>4075, dA[0.009,0.023] dB[0.

>4209, dA[0.337,0.010] dB[0.003,0.048] g[4.012,5.502]
>4210, dA[0.045,0.004] dB[0.002,0.016] g[4.484,5.117]
>4211, dA[0.056,0.008] dB[0.003,0.002] g[4.994,5.387]
>4212, dA[0.108,0.004] dB[0.005,0.002] g[4.703,4.595]
>4213, dA[0.199,0.009] dB[0.023,0.014] g[4.296,4.984]
>4214, dA[0.211,0.006] dB[0.182,0.079] g[4.055,4.512]
>4215, dA[0.029,0.099] dB[0.004,0.044] g[3.998,3.738]
>4216, dA[0.059,0.038] dB[0.009,0.013] g[5.019,4.899]
>4217, dA[0.005,0.090] dB[0.022,0.104] g[4.629,5.319]
>4218, dA[0.020,0.013] dB[0.020,0.169] g[4.693,5.400]
>4219, dA[0.003,0.038] dB[0.085,0.013] g[4.160,3.578]
>4220, dA[0.049,0.050] dB[0.209,0.000] g[4.906,5.255]
>4221, dA[0.061,0.023] dB[0.062,0.003] g[4.443,4.472]
>4222, dA[0.011,0.017] dB[0.020,0.002] g[4.227,4.111]
>4223, dA[0.003,0.018] dB[0.005,0.052] g[7.362,8.771]
>4224, dA[0.201,0.008] dB[0.056,0.032] g[5.587,6.073]
>4225, dA[0.017,0.016] dB[0.006,0.112] g[3.902,4.841]
>4226, dA[0.020,0.029] dB[0.058,0.067] g[4.075,4.411]
>4227, dA[0.009,0.143] dB[0.

>4361, dA[0.013,0.010] dB[0.002,0.161] g[5.978,6.689]
>4362, dA[0.007,0.079] dB[0.019,0.191] g[5.750,6.496]
>4363, dA[0.039,0.023] dB[0.054,0.017] g[5.259,5.251]
>4364, dA[0.019,0.023] dB[0.045,0.021] g[4.912,5.985]
>4365, dA[0.280,0.097] dB[0.017,0.007] g[4.324,5.679]
>4366, dA[0.004,0.059] dB[0.052,0.020] g[8.912,8.001]
>4367, dA[0.002,0.009] dB[0.015,0.061] g[5.473,5.890]
>4368, dA[0.010,0.005] dB[0.014,0.009] g[4.776,5.546]
>4369, dA[0.046,0.010] dB[0.142,0.010] g[4.191,4.846]
>4370, dA[0.003,0.009] dB[0.076,0.093] g[4.442,4.607]
>4371, dA[0.049,0.010] dB[0.005,0.030] g[3.750,4.491]
>4372, dA[0.005,0.009] dB[0.028,0.158] g[4.463,4.917]
>4373, dA[0.384,0.281] dB[0.037,0.001] g[4.202,4.680]
>4374, dA[0.005,0.082] dB[0.042,0.001] g[5.091,5.525]
>4375, dA[0.097,0.034] dB[0.112,0.010] g[5.202,5.065]
>4376, dA[0.034,0.023] dB[0.316,0.043] g[5.779,6.304]
>4377, dA[0.015,0.007] dB[0.044,0.014] g[6.307,6.347]
>4378, dA[0.023,0.011] dB[0.005,0.014] g[3.980,4.367]
>4379, dA[0.267,0.124] dB[0.